# Machine Learning

## Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Importar dados normalizados

In [2]:
dados = pd.read_csv('C:/Users/lucas/Documents/GitHub/INT/Manufatura Aditiva/Classificação Direção/Dados/Dados tratados.csv')
len(dados)

100

## Separando Treino e Teste

In [4]:
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### KNN

In [49]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
nrep = 100
accs = 0
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
for i in range(nrep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    knn = make_pipeline(StandardScaler(),
                        KNeighborsClassifier(n_neighbors=1))
    knn.fit(X_train, y_train)
    pred_knn = knn.predict(X_test)
    accs += accuracy_score(y_test,pred_knn)
    

print(accs/100)


0.9736666666666668


### SGD

In [32]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
nrep = 100
accs = 0
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
for i in range(nrep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    sgd = make_pipeline(StandardScaler(),
                        SGDClassifier(max_iter=1000, tol=1e-3))
    sgd.fit(X_train, y_train)
    pred_sgd = sgd.predict(X_test)
    accs += accuracy_score(y_test,pred_sgd)

print(accs/100)


0.9670000000000004


### SVC 

In [25]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
nrep = 100
accs = 0
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
for i in range(nrep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    svc = make_pipeline(StandardScaler(),
                        SVC())
    svc.fit(X_train,y_train)
    pred_svc = svc.predict(X_test)
    accs+=accuracy_score(y_test,pred_svc)

print(accs/100)

0.9933333333333336


### Árvore de Decisão

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

nrep = 100
accs = 0
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
for i in range(nrep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    tree = make_pipeline(StandardScaler(),
                        DecisionTreeClassifier())

    tree.fit(X_train,y_train)
    pred_tree = tree.predict(X_test)
    accs += accuracy_score(y_test,pred_tree)
print(accs/100)

0.9470000000000001


## K Means

In [55]:
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

nrep = 100
accs = 0
X = dados.drop(['Plano','Plano ID'],axis=1)
y = dados['Plano ID']
for i in range(nrep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    kmeans = make_pipeline(StandardScaler(),
                       KMeans(n_clusters=3))

    kmeans.fit(X_train,y_train)
    pred_kmeans = kmeans.predict(X_test)
    accs += accuracy_score(y_test,pred_kmeans)
print(accs/100)

0.31633333333333336
